In [1]:
import numpy as np

In [30]:
imfs = np.ones(5)

In [31]:
def check_imf_priors(imfs):
    imf_priors = np.zeros(len(imfs), dtype=bool)
    
    for ind,imf in enumerate(imfs):
        if not -10.0 < imf < 10:
            return False
    return True

In [32]:
check_imf_priors(imfs)

True

#### Fix the getParams function to work specifically for running IMFs

In [38]:
def get_fixed_params():
    maxi_fixed_params = {'zred': 3.548,
                         'mass': 1e8,
                         'logzsol': 0.0,
                         'dust2': 0.0,
                         'dust_type': 0}

    maxi_sfh_params = {'sfh': 1,
                       'tau': 0.05,
                       'tage': 1.0,
                       'fburst': 0.0,
                       'fage_burst': 0.0,
                       'tburst': 0.0,
                       'const': 0.0,
                       'sf_start': 0.0,
                       'sf_trunc': 0.0}

    
    params = {
              'imf_type': 5,
        
    return params

def init_prospect_generation():
    params = get_fixed_params()
    sps, wspec = getSps()
    obs = getObs()

    return params, obs, sps, wspec

#### Fix the getModel function to work specifically for running IMFs

In [37]:
def getModelIMF(**extras):
    from prospect.models import SpecModel
    from prospect.models.templates import TemplateLibrary

    params = TemplateLibrary['parametric_sfh']

    for key in list(params.keys()):
        params[key]['init'] = extras[key]

    model = SpecModel(params)

    return params, model

In [ ]:
def getModelSFH(add_burst=False, add_trunc=False, **extras):

    from prospect.models import SpecModel
    from prospect.models.templates import TemplateLibrary

    model_params = TemplateLibrary['parametric_sfh']

    if add_burst:
        model_params.update(TemplateLibrary['burst_sfh'])
        model_params['const'] = {'N': 1, 'isfree': False, 'init': 0.0, 'units': 'Solar Masses per year'}
        if add_trunc:
            model_params['sf_start'] = {'N': 1, 'isfree': False, 'init': 0.0, 'units': 'Gyrs'}
            model_params['sf_trunc'] = {'N': 1, 'isfree': False, 'init': 0.0, 'units': 'Gyrs'}

    for key in list(model_params.keys()):
        model_params[key]['init'] = extras[key]

    model = SpecModel(model_params)
    
    return model

In [34]:
from matplotlib.pyplot import *
import numpy as np

import sys
sys.path.append('/home/samantha/Documents/REU/scripts/') # Use the absolute path to the directory
from prospectFunctionsIMF import *

# from prospectFunctions import getObs, getWave, getParams, getModel, getSps
# import itertools
# import astropy.units as u
# import astropy.constants as const
# import time
# from tqdm import tqdm

In [ ]:
imf_type = 5 # Tabulated piece-wise power-law IMF

# imf_types = ['Salpeter (1955)', 'Chabrier (2003)', 'Kroupa (2001)', 'van Dokkum (2008)', 'Dave (2008)']

grid_ranges, run_params = getParams(imf_type=imf_type, len_logz=100, len_dust=6, len_tage=100, dust=True)
a = 1.0 + run_params['zred']

logzs = grid_ranges['logzsol']
dusts = grid_ranges['dust2']
tages = grid_ranges['tage']

num_models = len(logzs)*len(dusts)*len(tages)

obs = getObs(**run_params)
sps = getSps(**run_params)
wspec, wphot = getWave(obs=obs, sps=sps, **run_params)

In [ ]:
specs = []

run_params['imf_type'] = imf_type
run_params['logzsol'] = logzs[0]
run_params['dust2'] = dusts[0]
run_params['tage'] = tages[0]

test_imfs = ['0.08 0.50 1.3\n0.50 1.00 2.3\n1.00 120. 2.3\n',
             '0.008 0.50 1.3\n0.50 1.00 2.3\n1.00 120. 2.3\n',
             '0.08 0.50 1.3\n0.50 1.00 2.3\n1.00 300. 2.3\n',
             '0.008 0.50 1.3\n0.50 1.00 2.3\n1.00 300. 2.3\n']

for test in tqdm(test_imfs):
    with open('/home/samantha/fsps/data/imf.dat', 'w') as f:
        f.write(test) # <- content ^

    with open('/home/samantha/fsps/data/imf.dat', 'r') as f:
        content = f.read()
        print(content)
        print()

    sps = getSps(**run_params)

    model = getModel(**run_params)
    init_spec, init_phot, init_mfrac = model.predict(model.theta, obs=obs, sps=sps)

    specs.append(init_spec)